## Generate all Combiantions

In [6]:
import lal

In [2]:
rt = lal.graphs.from_head_vector_to_rooted_tree(heads)
lal.linarr.chunk_syntactic_dependency_tree(rt, chunk_type)

Signature: lal.linarr.chunk_syntactic_dependency_tree(*args) -> 'lal::graphs::rooted_tree'
Docstring:
Overloaded function
-------------------
* `chunk_syntactic_dependency_tree(rt, algo)`  
    lal::linarr::chunk_syntactic_dependency_tree
    Chunks a syntactic dependency tree according to the algorithm passed as
    parameter.  

    This function assumes the identity arrangement to chunk this tree.  

    Parameters:


    * `rt` :  
        Input syntactic dependency tree.  
    * `algo` :  
        Algorithm of choice.  

    Returns:
    A syntactic dependency tree result of the chosen chunking algorithm.  

* `chunk_syntactic_dependency_tree(rt, arr, algo)`  
    lal::linarr::chunk_syntactic_dependency_tree
    Chunks a syntactic dependency tree according to the algorithm passed as
    parameter.  

    Parameters:


    * `rt` :  
        Input syntactic dependency tree.  
    * `arr` :  
        Non-empty input linear arrangement.  
    * `algo` :  
        Algorithm of choice.

In [7]:
def is_projective(rt):
       if lal.linarr.num_crossings(rt) > 0: return False
       r = rt.get_root()
       edges = rt.get_edges()
       root_covered = any([((u<r and r<v) or (v<r and r<u)) for (u,v) in edges])
       return not root_covered

## Draw dependency graphs
Print them to tex to show grouping in chunks

In [8]:
import numpy as np
import rpy2.robjects as ro

In [13]:
# to call R function :)
r = ro.r
r['source']('code_chunks.R')
assign_chunks_r = ro.globalenv['assign_chunks']

In [16]:
# fixed rows
fix_header_2 = r'\begin{deptext}[column sep = .5cm]'
fix_middle   = r'\end{deptext}'
fix_end      = r'\end{dependency}'

# variable rows
def header_1(chunk_type): return r'\begin{dependency}[theme = simple, group style =' + chunk_type + ']'

def draw_labels(labels): return r' \& '.join(str(el) for el in labels) + r' \\'

def draw_group(start,end): return '\wordgroup{1' + '}{' + str(start) + '}{' + str(end) + r'}{}' 

def draw_edge(head,dep): return '\depedge{' + str(head) + '}{' + str(dep) + r'}{}'

def draw_more_groups(memberships):
    # create and concatenate groups from draw_group and memberships
    groups = np.unique(memberships)
    
    start_end_pairs = []
    for group in groups:
        start = memberships.index(group) + 1
        end = start + memberships.count(group) - 1
        start_end_pairs.append(tuple((start,end)))

    return '\n'.join(draw_group(limits[0],limits[1]) for i,limits in enumerate(start_end_pairs))

def draw_more_edges(heads):
    # find root
    root_pos = heads.index(0) + 1
    root_tex = '\deproot{' + str(root_pos) + r'}{}' 

    # find edges
    head_dep_pairs = []
    for i,head in enumerate(heads):
        if head != 0:
            dep = i + 1
            head_dep_pairs.append(tuple((head,dep)))

    edges_tex = '\n'.join(draw_edge(edges[0],edges[1]) for i,edges in enumerate(head_dep_pairs))

    return '\n'.join([root_tex,edges_tex])

def draw_graph(heads,labels,chunk_type):
    var_header_1 = header_1(chunk_type)
    # find memberships
    heads_r = ro.IntVector(heads)
    memberships = list(assign_chunks_r(heads_r,chunk_type))
    # transform labels
    labels = draw_labels(labels)
    # add edges
    edges = draw_more_edges(heads)
    # create and concatenate groups from draw_groups and memberships
    groups = draw_more_groups(memberships)
    
    return var_header_1 + '\n' + fix_header_2 + '\n' + labels + '\n' + fix_middle + '\n' + edges + '\n' + groups + '\n' + fix_end

def draw_graph(heads,labels,chunk_type):
    var_header_1 = header_1(chunk_type)
    # find memberships
    heads_r = ro.IntVector(heads)
    memberships = list(assign_chunks_r(heads_r,chunk_type))
    # transform labels
    labels = draw_labels(labels)
    # add edges
    edges = draw_more_edges(heads)
    # create and concatenate groups from draw_groups and memberships
    groups = draw_more_groups(memberships)
    
    return var_header_1 + '\n' + fix_header_2 + '\n' + labels + '\n' + fix_middle + '\n' + edges + '\n' + groups + '\n' + fix_end



def all_arrangements(n):
    arrs_dict = {}
    # generate all projective arrangements
    Gen = lal.generate.all_ulab_rooted_trees(n)
    while not Gen.end():
        rt = Gen.yield_tree()
        if is_projective(rt):
            Gen_arr = lal.generate.all_projective_arrangements(rt)
            while not Gen_arr.end():
                arr = Gen_arr.yield_arrangement() 
                labels = arr.inverse_as_vector()
                heads = rt.get_head_vector(arr)
                arrs_dict[heads] = labels
                
    return arrs_dict

# put all in a latex file!
fix_fig_beg    = r'\begin{figure}[H]'
fix_mini_beg   = r'\begin{minipage}[b]{0.3\textwidth} \centering'
fix_mini_end   = r'\end{minipage}\hfill'
fix_fig_end    = r'\end{figure}'

def write_graph_to_tex(graphs,filename):
    with open(f'graphs/{filename}.tex','w') as file:
        file.write(graphs)

def print_file(n,chunk_type):
    # all arrangement of n
    arr_dict = all_arrangements(n)
    # create all graphs
    mini_pages = []
    for heads,labels in arr_dict.items():
        graph = draw_graph(heads,labels,chunk_type)
        mini_pages.append(fix_mini_beg + graph + fix_mini_end)
    # join all graphs
    if len(mini_pages) > 15:
        graphs1 = '\n'.join(mini_pages[:15])
        graphs2 = '\n'.join(mini_pages[15:])
        graphs_tex = fix_fig_beg + '\n' + graphs1 + '\n' + fix_fig_end + '\n' + fix_fig_beg + '\n' + graphs2 + '\n' + fix_fig_end
    else:
        graphs = '\n'.join(mini_pages)
        graphs_tex = fix_fig_beg + '\n' + graphs + '\n' + fix_fig_end

    # write to latex
    filename = str(n) + chunk_type
    write_graph_to_tex(graphs_tex,filename)

All arrangements

In [4]:
#chunk_types = ['anderson','makutec']
#ns=[3,4]

#for chunk_type in chunk_types:
#    for n in ns:
#        print_file(n,chunk_type)


In [17]:
# example
heads = [2, 4, 4, 0, 6, 4, 4, 7, 10, 4, 10, 11, 12, 19, 14, 15, 16, 17, 13, 23, 20, 21, 19, 26, 26, 23]
labels = range(len(heads))


graph = draw_graph(heads,labels,'anderson')
filename = 'test'
write_graph_to_tex(graph,filename)

In [18]:
# example

heads = [0,1,7,5,7,7,1,7,8,1,14,14,14,10,14,17,15]
chunk_types = ['anderson','macutek']
labels = 'Ask not what your country can do for you, ask what you can do for your country.'
labels = labels.split(' ')

for chunk_type in chunk_types:
    graph = draw_graph(heads,labels,chunk_type)
    filename = 'thisisanexample_{}'.format(chunk_type)
    write_graph_to_tex(graph,filename)

In [21]:
# Lluís Algorithm

heads = [0,1,7,5,7,7,1,7,8,1,14,14,14,10,14,17,15]
chunk_types = ['Anderson','Macutek']
#labels = 'Ask not what your country can do for you, ask what you can do for your country.'
#labels = labels.split(' ')
rt = lal.graphs.from_head_vector_to_rooted_tree(heads)
lal.linarr.chunk_syntactic_dependency_tree(rt, labels,chunk_types[0])

TypeError: Wrong number or type of arguments for overloaded function 'chunk_syntactic_dependency_tree'.
  Possible C/C++ prototypes are:
    lal::linarr::chunk_syntactic_dependency_tree(lal::graphs::rooted_tree const &,lal::linarr::algorithms_chunking const &)
    lal::linarr::chunk_syntactic_dependency_tree(lal::graphs::rooted_tree const &,lal::linear_arrangement const &,lal::linarr::algorithms_chunking const &)
